In [23]:
import pandas as pd
import json
import os
import re

In [2]:
DATA_PATH = 'data/api_fetches/'
WEATHER_PREFIX = 'boston_weather_'
PRED_PREFIX = 'predictions_'
LOC_PREFIX = 'vehicle_locations_'
pred_time = '20240605_153001'
loc_time = '20240605_153001'
weather_time = '20240605_15'
pred_path = DATA_PATH + PRED_PREFIX + pred_time + '.json'
loc_path = DATA_PATH + LOC_PREFIX + loc_time + '.json'
weather_path = DATA_PATH + WEATHER_PREFIX + weather_time + '.json'

In [3]:
predictions = pd.read_json(pred_path)
locations = pd.read_json(loc_path)
with open(weather_path, 'r') as file:
    weather_data = json.load(file)
weather_data = [weather_data]
weather = pd.DataFrame(weather_data)


In [4]:
def clean_predictions(df):
    df = df.explode('stop_time_updates')
    df = df.join(pd.json_normalize(df['stop_time_updates']))
    df = df.drop(columns=['stop_time_updates'])
    return df

In [35]:
def combine(predictions_df, locations_df, weather_df, timestamp):
    df = pd.merge(predictions_df, locations_df, on='trip_id', how='inner')
    df['collection_time'] = timestamp
    weather_df = weather_df.drop('timestamp', axis=1)
    weather_df['collection_time'] = timestamp
    df = pd.merge(df, weather_df, on='collection_time')
    return df

In [ ]:
predictions = clean_predictions(predictions)

In [21]:
full_df = combine(predictions, locations, weather, '20240605_153000')

In [22]:
full_df

,trip_id,start_time,start_date,stop_id,arrival,departure,vehicle_id,latitude,longitude,bearing,timestamp,collection_time,precipitationProbability,iceAccumulation,rainAccumulation,sleetAccumulation,snowAccumulation,snowDepth,temperature
0,61392224,2024-06-06 15:00:00,20240605,5563,1.717619e+09,1.717619e+09,y1911,42.335201,-71.090225,188,2024-06-05 19:29:58,20240605_153000,0,0,0,0,0,0,29.31
1,CR-670369-943,2024-06-06 15:17:00,20240605,45406,1.717619e+09,1.717619e+09,1846,42.310535,-71.072281,199,2024-06-05 19:29:39,20240605_153000,0,0,0,0,0,0,29.31
2,CR-670369-943,2024-06-06 15:17:00,20240605,45406,1.717619e+09,1.717619e+09,1846,42.310535,-71.072281,199,2024-06-05 19:29:39,20240605_153000,0,0,0,0,0,0,29.31
3,CR-670369-943,2024-06-06 15:17:00,20240605,45406,1.717619e+09,1.717619e+09,1846,42.310535,-71.072281,199,2024-06-05 19:29:39,20240605_153000,0,0,0,0,0,0,29.31
4,CR-670369-943,2024-06-06 15:17:00,20240605,45406,1.717619e+09,1.717619e+09,1846,42.310535,-71.072281,199,2024-06-05 19:29:39,20240605_153000,0,0,0,0,0,0,29.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11746,61869229,2024-06-06 15:02:00,20240605,10642,1.717621e+09,NaN,y0882,42.423260,-71.090836,51,2024-06-05 19:29:52,20240605_153000,0,0,0,0,0,0,29.31
11747,61869229,2024-06-06 15:02:00,20240605,10642,1.717621e+09,NaN,y0882,42.423260,-71.090836,51,2024-06-05 19:29:52,20240605_153000,0,0,0,0,0,0,29.31
11748,61869229,2024-06-06 15:02:00,20240605,10642,1.717621e+09,NaN,y0882,42.423260,-71.090836,51,2024-06-05 19:29:52,20240605_153000,0,0,0,0,0,0,29.31
11749,61869229,2024-06-06 15:02:00,20240605,10642,1.717621e+09,NaN,y0882,42.423260,-71.090836,51,2024-06-05 19:29:52,20240605_153000,0,0,0,0,0,0,29.31


In [24]:
def rename_files_in_directory(directory):
    # Pattern to match the files with a full timestamp including seconds
    full_timestamp_pattern = re.compile(r"^(.*?_\d{8}_\d{4})(\d{2})(\.json)$")
    # Pattern to match the files with only hours in the timestamp
    hour_only_pattern = re.compile(r"^(.*?_\d{8}_\d{2})(\.json)$")

    for filename in os.listdir(directory):
        full_match = full_timestamp_pattern.match(filename)
        hour_match = hour_only_pattern.match(filename)

        if full_match:
            # Renaming files with a full timestamp to set seconds to '00'
            new_filename = f"{full_match.group(1)}00{full_match.group(3)}"
            old_filepath = os.path.join(directory, filename)
            new_filepath = os.path.join(directory, new_filename)
            os.rename(old_filepath, new_filepath)
            print(f"Renamed: {old_filepath} -> {new_filepath}")

        elif hour_match:
            # Renaming files with hours only to add '0000' for minutes and seconds
            new_filename = f"{hour_match.group(1)}0000{hour_match.group(2)}"
            old_filepath = os.path.join(directory, filename)
            new_filepath = os.path.join(directory, new_filename)
            os.rename(old_filepath, new_filepath)
            print(f"Renamed: {old_filepath} -> {new_filepath}")

In [26]:
rename_files_in_directory('data/api_fetches/20240605')

Renamed: data/api_fetches/2024-06-05/vehicle_locations_20240605_222901.json -> data/api_fetches/2024-06-05/vehicle_locations_20240605_222900.json
Renamed: data/api_fetches/2024-06-05/predictions_20240605_201429.json -> data/api_fetches/2024-06-05/predictions_20240605_201400.json
Renamed: data/api_fetches/2024-06-05/vehicle_locations_20240605_232901.json -> data/api_fetches/2024-06-05/vehicle_locations_20240605_232900.json
Renamed: data/api_fetches/2024-06-05/predictions_20240605_183901.json -> data/api_fetches/2024-06-05/predictions_20240605_183900.json
Renamed: data/api_fetches/2024-06-05/vehicle_locations_20240605_225401.json -> data/api_fetches/2024-06-05/vehicle_locations_20240605_225400.json
Renamed: data/api_fetches/2024-06-05/predictions_20240605_175101.json -> data/api_fetches/2024-06-05/predictions_20240605_175100.json
Renamed: data/api_fetches/2024-06-05/predictions_20240605_165101.json -> data/api_fetches/2024-06-05/predictions_20240605_165100.json
Renamed: data/api_fetches/

In [69]:
def aggregate(date, save_to_parquet=False):
    df_list = []
    counter = 0
    for hour in range(15, 24):
        for minute in range(0, 60):
            time = f'{hour:02}{minute:02}00'
            timestamp = f'{date}_{time}'
            weather_timestamp = f'{date}_{hour:02}0000'
            
            predictions_path = f'{DATA_PATH}{date}/{PRED_PREFIX}{timestamp}.json'
            locations_path = f'{DATA_PATH}{date}/{LOC_PREFIX}{timestamp}.json'
            weather_path = f'{DATA_PATH}{date}/{WEATHER_PREFIX}{weather_timestamp}.json'
            
            try:
                predictions = pd.read_json(predictions_path)
                locations = pd.read_json(locations_path)
                with open(weather_path, 'r') as file:
                    weather = json.load(file)
                weather = [weather]
                weather = pd.DataFrame(weather)
                predictions = clean_predictions(predictions)
                
                min_df = combine(predictions, locations, weather, timestamp)
                df_list.append(min_df)
            except:
                print(f'Error with {timestamp}')
            
    df = pd.concat(df_list)
    df['start_time'] = pd.to_datetime(df['start_time'], errors='coerce')
    df.to_parquet(f'{DATA_PATH}{date}.parquet')
    return df
            